# core

> test

# | default_exp core

In [ ]:
# | hide
from nbdev.showdoc import *

from bertopic import BERTopic
from datetime import datetime
from joblib import load as joblibload
import matplotlib.pyplot as plt
import mlflow
from mlflow.models.signature import infer_signature

# import numpy as np
import pandas as pd
from sklearn.base import TransformerMixin
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfTransformer,
    TfidfVectorizer,
)
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# import spacy
# import en_core_web_sm
# from spacy.lang.en.stop_words import STOP_WORDS

# from spacy.lemmatizer import Lemmatizer
from tqdm import tqdm
from typing import Any

In [ ]:
raw_data_path = '../data/recipes-en-201706/epicurious-recipes_m2.json'

joblib_basepath = '../joblib/2022.08.23/'

cv_path = joblib_basepath + 'countvec.joblib'
tfidf_path = joblib_basepath + 'tfidf.joblib'
full_df_path = joblib_basepath + 'recipes_with_cv.joblib'
reduced_df_path = joblib_basepath + 'reduced_df.joblib'

In [ ]:
full_df = joblibload(full_df_path)
full_df

,id,dek,hed,aggregateRating,ingredients,prepSteps,reviewsCount,willMakeAgainPct,cuisine_name,photo_filename,...,zest pith,zest vegetable,zinfandel,ziti,zucchini,zucchini blossom,zucchini crookneck,zucchini squash,árbol,árbol pepper
0,54a2b6b019925f464b373351,How does fried chicken achieve No. 1 status? B...,Pickle-Brined Fried Chicken,3.11,"[1 tablespoons yellow mustard seeds, 1 tablesp...",[Toast mustard and coriander seeds in a dry me...,7,100,Missing Cuisine,51247610_fried-chicken_1x1.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,54a408a019925f464b3733bc,Spinaci all'Ebraica,Spinach Jewish Style,3.22,"[3 pounds small-leaved bulk spinach, Salt, 1/2...",[Remove the stems and roots from the spinach. ...,5,80,Italian,EP_12162015_placeholders_rustic.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,54a408a26529d92b2c003631,"This majestic, moist, and richly spiced honey ...",New Year’s Honey Cake,3.62,"[3 1/2 cups all-purpose flour, 1 tablespoon ba...",[I like this cake best baked in a 9-inch angel...,105,88,Kosher,EP_09022015_honeycake-2.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,54a408a66529d92b2c003638,The idea for this sandwich came to me when my ...,The B.L.A.Bagel with Lox and Avocado,4.00,"[1 small ripe avocado, preferably Hass (see No...","[A short time before serving, mash avocado and...",7,100,Kosher,EP_12162015_placeholders_casual.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,54a408a719925f464b3733cc,"In 1930, Simon Agranat, the chief justice of t...",Shakshuka a la Doktor Shakshuka,2.71,"[2 pounds fresh tomatoes, unpeeled and cut in ...","[1. Place the tomatoes, garlic, salt, paprika,...",7,83,Kosher,EP_12162015_placeholders_formal.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34751,59541a31bff3052847ae2107,Buttering the bread before you waffle it ensur...,Waffled Ham and Cheese Melt with Maple Butter,0.00,"[1 tablespoon unsalted butter, at room tempera...","[Preheat the waffle iron on low., Spread a thi...",0,0,Missing Cuisine,waffle-ham-and-cheese-melt-062817.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34752,5954233ad52ca90dc28200e7,"Spread this easy compound butter on waffles, p...",Maple Butter,0.00,"[8 tablespoons (1 stick) salted butter, at roo...",[Combine the ingredients in a medium-size bowl...,0,0,Missing Cuisine,EP_12162015_placeholders_bright.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34753,595424c2109c972493636f83,Leftover mac and cheese is not exactly one of ...,Waffled Macaroni and Cheese,0.00,"[3 tablespoons unsalted butter, plus more for ...",[Preheat the oven to 375°F. Butter a 9x5-inch ...,0,0,Missing Cuisine,waffle-mac-n-cheese-062816.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34754,5956638625dc3d1d829b7166,A classic Mexican beer cocktail you can sip al...,Classic Michelada,0.00,"[Coarse salt, 2 lime wedges, 2 ounces tomato j...",[Place about 1/4 cup salt on a small plate. Ru...,0,0,Missing Cuisine,Classic Michelada 07292017.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
full_df['prepSteps']

0        [Toast mustard and coriander seeds in a dry me...
1        [Remove the stems and roots from the spinach. ...
2        [I like this cake best baked in a 9-inch angel...
3        [A short time before serving, mash avocado and...
4        [1. Place the tomatoes, garlic, salt, paprika,...
                               ...                        
34751    [Preheat the waffle iron on low., Spread a thi...
34752    [Combine the ingredients in a medium-size bowl...
34753    [Preheat the oven to 375°F. Butter a 9x5-inch ...
34754    [Place about 1/4 cup salt on a small plate. Ru...
34755    [Combine the water, honey, rosemary, and grape...
Name: prepSteps, Length: 34656, dtype: object

In [ ]:
full_df['prepSteps'][0]

['Toast mustard and coriander seeds in a dry medium saucepan over medium heat, tossing often, until mustard seeds begin to pop, about 3 minutes. Add vinegar, salt, and sugar and bring to a boil. Reduce heat and simmer, stirring often, until salt and sugar are dissolved, about 4 minutes. Remove from heat; stir in dill and 4 cups water. Let cool.',
 'Place chicken and brine in a large resealable plastic bag; chill 3 hours. Remove chicken from brine, scraping off seeds, cover, and chill until ready to fry.',
 'Fit a large pot with thermometer and pour in oil to measure 2". Heat over medium-high heat until thermometer registers 350°F.',
 'Meanwhile, place buttermilk in a large bowl. Place flour in another large bowl; season with kosher salt. Working in batches, coat chicken in buttermilk, then dredge in flour, dipping your fingers in buttermilk as you pack flour on to help create moistened, shaggy bits (the makings of a super-crisp crust); transfer to a baking sheet.',
 'Working in batches

In [ ]:
recipe_steps = full_df['prepSteps'].apply(" ".join).str.lower()

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(recipe_steps)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,9127,-1_chicken_the_sugar_cake,"[chicken, the, sugar, cake, in, and, of, until...",[in a bowl with a fork stir together the crumb...
1,0,672,0_pork_chops_meat_roast,"[pork, chops, meat, roast, marinade, fat, sauc...",[place pork and vinaigrette in pie dish; turn ...
2,1,484,1_lamb_chops_mediumrare_shanks,"[lamb, chops, mediumrare, shanks, meat, marina...",[season lamb chops with salt and pepper. heat ...
3,2,449,2_fish_fillets_fillet_opaque,"[fish, fillets, fillet, opaque, cooked, sauce,...","[combine cucumber, onions, cilantro, 4 1/2 tab..."
4,3,447,3_shrimp_cooked_opaque_through,"[shrimp, cooked, opaque, through, sauce, saut,...",[heat 1 tablespoon oil in heavy large saucepan...
...,...,...,...,...,...
430,429,10,429_ice_chocolate_freeze_ganache,"[ice, chocolate, freeze, ganache, cream, cooki...",[preheat oven to 350°f. combine first 6 ingred...
431,430,10,430_shortcakes_shortcake_blackberry_stars,"[shortcakes, shortcake, blackberry, stars, str...",[position rack in center of oven and preheat t...
432,431,10,431_rice_nori_fluff_rose,"[rice, nori, fluff, rose, jasmine, date, sesam...",[into a large saucepan of salted boiling water...
433,432,10,432_stock_skimming_bones_froth,"[stock, skimming, bones, froth, trimmings, wel...",[in a well-buttered heavy saucepan combine the...


# | export
def foo():
    pass

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()